In [ ]:
import ipywidgets
import io
import IPython.display
import ffmpeg
import matplotlib
import numpy as np
import PIL
import tensorflow as tf
from contextlib import closing
from contextlib import contextmanager
from IPython.display import display
from ipywidgets import interact
from jupyter_renderer_widget import PyplotRenderer
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
class Linear(tf.keras.Model):
    def __init__(self):
        super(Linear, self).__init__()
        self.W = tf.Variable(5., name='weight')
        self.B = tf.Variable(10., name='bias')

    def call(self, inputs):
        return inputs * self.W + self.B

In [ ]:
train_niterations = 5000
train_nsamples = 40
train_xs = tf.random.uniform([train_nsamples]) * 10
noise = tf.random.normal([train_nsamples])
train_ys = train_xs * 3 + 2 + noise

In [ ]:
def get_loss(model, xs, ys):
    error = model(xs) - ys
    return tf.reduce_mean(tf.square(error))

model = Linear()
weight_snapshots = [model.get_weights()]
loss_snapshots = [get_loss(model, train_xs, train_ys)]
print("Initial loss: {:.3f}".format(loss_snapshots[0]))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
for i in range(train_niterations):
    with tf.GradientTape() as tape:
        loss_value = get_loss(model, train_xs, train_ys)
    grads = tape.gradient(loss_value, model.weights)
    optimizer.apply_gradients(zip(grads, model.weights))
    loss_snapshots.append(loss_value)
    weight_snapshots.append(model.get_weights())
    
print("Final loss: {:.3f}".format(loss_snapshots[-1]))

In [ ]:
sub_snapshots = weight_snapshots[::40]

def render(ax, frame_num):
    ax.clear()
    weights = sub_snapshots[frame_num]
    model.set_weights(weights)
    model_xs = tf.constant([0., 10.])
    model_ys = model(model_xs)
    ax.plot(model_xs.numpy(), model_ys.numpy())
    ax.plot(train_xs.numpy(), train_ys.numpy(), 'go')
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 40)

frame_count = len(sub_snapshots)
display(PyplotRenderer(render, frame_count))